# Big G Express - Data Exploration

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingClassifier

from imblearn.over_sampling import SMOTE

from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score

from joblib import dump, load

In [54]:
faults = pd.read_pickle('../data/faults_filtered.pkl')
#y_derate = pd.read_pickle('../data/target_derate.pkl') # this one is the base model, 6 hr
#y_derate = pd.read_pickle('../data/target_derate3h.pkl')
#y_derate = pd.read_pickle('../data/target_derate12h.pkl')
y_derate = pd.read_pickle('../data/target_derate24h.pkl')
#y_75derate = pd.read_pickle('../data/target_75derate.pkl')
diagnostics_imputed = pd.read_pickle('../data/diagnostics_imputed.pkl')

In [55]:
# this one is mostly NaNs, just 250 values or so
diagnostics_imputed = diagnostics_imputed.drop(columns='ServiceDistance')

# and this drops columns that are not useful for predictions
faults = faults.drop(columns=['ESS_Id', 'active', 'eventDescription','ecuSoftwareVersion', 'ecuSerialNumber', 
    'ecuModel', 'ecuMake', 'ecuSource', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp'])

Remember there are parts of columns (where a particular truck had no values)

In [56]:
# this was just a simple fill with mean..
diagnostics_imputed['AcceleratorPedal'] = diagnostics_imputed['AcceleratorPedal'].fillna(value=diagnostics_imputed['AcceleratorPedal'].mean())
diagnostics_imputed['CruiseControlSetSpeed'] = diagnostics_imputed['CruiseControlSetSpeed'].fillna(value=diagnostics_imputed['CruiseControlSetSpeed'].mean())
diagnostics_imputed['EngineTimeLtd'] = diagnostics_imputed['EngineTimeLtd'].fillna(value=diagnostics_imputed['EngineTimeLtd'].mean())
diagnostics_imputed['FuelLevel'] = diagnostics_imputed['FuelLevel'].fillna(value=diagnostics_imputed['FuelLevel'].mean())
diagnostics_imputed['FuelTemperature'] = diagnostics_imputed['FuelTemperature'].fillna(value=diagnostics_imputed['FuelTemperature'].mean())
diagnostics_imputed['SwitchedBatteryVoltage'] = diagnostics_imputed['SwitchedBatteryVoltage'].fillna(value=diagnostics_imputed['SwitchedBatteryVoltage'].mean())
diagnostics_imputed['Throttle'] = diagnostics_imputed['Throttle'].fillna(value=diagnostics_imputed['Throttle'].mean())

In [32]:
# this took 30 min and didn't stop ...
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer, KNNImputer
# scaler = StandardScaler().fit(diagnostics_imputed)

# knn_filled = scaler.inverse_transform(KNNImputer().fit_transform(scaler.transform(diagnostics_imputed)))

# diagnostics_imputed = IterativeImputer().fit_transform(diagnostics_imputed)

## Merging and prepping the data

In [ ]:
faults_diagnostics = faults.merge(diagnostics_imputed, left_on='RecordID', right_on='FaultId', how='inner')

In [ ]:
faults_diagnostics['spn_fmi'] = ['_'.join(i) for i in zip(faults_diagnostics['spn'].astype(str), faults_diagnostics['fmi'].astype(str))]

faults_diagnostics = pd.get_dummies(faults_diagnostics, columns=['spn_fmi'], prefix='spn_fmi')

faults_diagnostics = faults_diagnostics.sort_values(by=['EquipmentID', 'EventTimeStamp'])

In [ ]:
# to obtain the one hot encoded columns since there are so many
faults_cols = ['EventTimeStamp'] + [col for col in faults_diagnostics.columns if 'spn_fmi' in col] 

diagnostics_cols = ['EventTimeStamp', 'activeTransitionCount', 'AcceleratorPedal',
         'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
         'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
        'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
        'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus',
        'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

In [ ]:
faults_rolling = (
    faults_diagnostics
    .groupby('EquipmentID')[faults_cols]
    .rolling(window = '1d', on = "EventTimeStamp")
    .max()
)

faults_rolling = faults_rolling.reset_index()

In [ ]:
diagnostics_rolling = (
    faults_diagnostics
    .groupby('EquipmentID')[diagnostics_cols]
    .rolling(window = '1d', on = "EventTimeStamp")
    .mean()
)

diagnostics_rolling = diagnostics_rolling.reset_index()

In [ ]:
faults_rolling = pd.merge(faults_diagnostics['RecordID'], #[['RecordID'] + diagnostics_cols]
                          faults_rolling,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

diagnostics_rolling = pd.merge(faults_diagnostics['RecordID'],
                          diagnostics_rolling,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

In [ ]:
faults_diagnostics_rolling =  pd.merge(diagnostics_rolling.drop(columns=['EquipmentID', 'EventTimeStamp']),
            faults_rolling.drop(columns=['EquipmentID', 'EventTimeStamp']),
            on = 'RecordID')

In [ ]:
faults_diagnostics_rolling = faults_diagnostics_rolling.sort_values('RecordID').drop(columns='RecordID')

In [ ]:
#faults_diagnostics_rolling = faults_rolling.drop(columns=['RecordID', 'EquipmentID', 'EventTimeStamp_x', 'EventTimeStamp_y'])


## Training and test

In [ ]:
# use stratify on target (with derate) and split based on trucks
X_train, X_test, y_train, y_test = train_test_split(faults_diagnostics_rolling, y_derate.sort_values('RecordID')['target'], train_size = 0.8, test_size = 0.2, random_state = 42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
gbr = Pipeline(
    steps = [
        ('gb', GradientBoostingClassifier(verbose=True)) #n_estimators = 1000, learning_rate=0.01
    ]
)

gbr.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_train, gbr.predict(X_train))

In [ ]:
print(classification_report(y_train, gbr.predict(X_train)))

In [ ]:
importances = pd.DataFrame({
    'variable': gbr.feature_names_in_,
    'importance': gbr['gb'].feature_importances_
})

importances.sort_values('importance', ascending = False).head(20)

NOTE: during one of the trainings there was a particular spn-fmi code that got to the top, with importance of 0.8! It was the 46262 code and after inspecting, despite appearing only once in the dataset, since there are many events happening in a smlal timeframe around it, it got picked up as important!

In [ ]:
oversampler = SMOTE(k_neighbors=5, random_state=42)

In [ ]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [ ]:
y_smote.value_counts()

In [ ]:
gbr_smoted = Pipeline(
    steps = [
        ('gb', GradientBoostingClassifier(verbose=True))#n_estimators = 1000, learning_rate=0.01
    ]
)

gbr_smoted.fit(X_smote, y_smote)

In [ ]:
y_predict = gbr_smoted.predict(X_train)

In [ ]:
confusion_matrix(y_train, y_predict)

In [ ]:
confusion_matrix(y_test, gbr.predict(X_test))

In [ ]:
confusion_matrix(y_test, gbr_smoted.predict(X_test))

In [ ]:
print(classification_report(y_train, y_predict))

In [ ]:
importances = pd.DataFrame({
    'variable': gbr_smoted.feature_names_in_,
    'importance': gbr_smoted['gb'].feature_importances_
})

importances.sort_values('importance', ascending = False).head(25)

In [ ]:
gbr.predict_proba(X_test)[:,1]

In [ ]:
roc_auc_score(y_true=y_test, y_score=gbr.predict_proba(X_test)[:,1])

In [ ]:
roc_auc_score(y_true=y_test, y_score=gbr_smoted.predict_proba(X_test)[:,1])

## Better Train-Test split

The above is a good start. However, there are trucks whose events end up mixed between both train and test split. Instead, we want to make sure that each individual truck only appears in one.

In [57]:
print(faults['EquipmentID'].nunique())
print(faults.loc[faults['spn'] == 5246]['EquipmentID'].nunique())

1042
189


First off, get the two lists of trucks that had (or not) a full derate.

In [58]:
all_trucks = faults['EquipmentID'].unique()
derate_trucks = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique()
no_derate_trucks = all_trucks[np.isin(all_trucks, derate_trucks, invert=True)]

Secondly, put those lists together, marking if a derate occured (1) or not (0).

In [59]:
trucks_df = pd.concat([
            pd.DataFrame({'EquipmentID': derate_trucks, 'derate': 1}),
            pd.DataFrame({'EquipmentID': no_derate_trucks, 'derate': 0}) 
            ])

Lastly, use the train_test_split, by accounting for the proportion of 'derates' in both (using stratify)

In [60]:
trucks_train, trucks_test = train_test_split(trucks_df, stratify=trucks_df['derate'], train_size = 0.8, test_size = 0.2, random_state = 42)

In [61]:
# this was just to verify that the proportions of trucks with and without derate in two samples are equal
# print(trucks_train['derate'].value_counts(normalize=True))
# print(trucks_test['derate'].value_counts(normalize=True))

# print(faults.loc[faults['EquipmentID'].isin(trucks_train['EquipmentID'])].shape[0])
# print(faults.loc[faults['EquipmentID'].isin(trucks_test['EquipmentID'])].shape[0])

Finally, use that information to split the diagnostics and targets.

In [62]:
# need to extract this because the train dataset only has RecordID
records_train = faults.loc[faults['EquipmentID'].isin(trucks_train['EquipmentID'])]['RecordID']
records_test = faults.loc[faults['EquipmentID'].isin(trucks_test['EquipmentID'])]['RecordID']

In [63]:
y_train = y_derate.loc[y_derate['RecordID'].isin(records_train)].sort_values('RecordID').drop(columns='RecordID')['target']
y_test = y_derate.loc[y_derate['RecordID'].isin(records_test)].sort_values('RecordID').drop(columns='RecordID')['target']

Now that the y_train and y_test are sorted, time to do the same for the X_train and X_test.

In [64]:
faults_diagnostics = faults.merge(diagnostics_imputed, left_on='RecordID', right_on='FaultId', how='inner').drop(columns='FaultId')

Next it depends on how these get prepared, so I'll build a function. It takes the faults + diagnostic con

In [65]:
def windowize_predictors(fulldetail_faults, time_window='1d', faults_agg='max', windowize_diagnostics = True, diagnostics_agg='mean'):

    # pull out the diagnostics table columns for later
    diagnostics_cols = [col for col in fulldetail_faults.columns if col not in ['RecordID', 'spn', 'fmi', 'EquipmentID']]

    # create a combined spn_fmi column to make dummies out of
    fulldetail_faults['spn_fmi'] = ['_'.join(i) for i in zip(fulldetail_faults['spn'].astype(str), fulldetail_faults['fmi'].astype(str))]

    # make dummies (one hot encode)
    fulldetail_faults = pd.get_dummies(fulldetail_faults, columns=['spn_fmi'], prefix='spn_fmi')

    # make sure the dataframe is in the right order to be able to later re-assign RecordID to it
    fulldetail_faults = fulldetail_faults.sort_values(by=['EquipmentID', 'EventTimeStamp'])

    # pull out all the Faults table columns (now one hot encoded)
    faults_cols = ['EventTimeStamp'] + [col for col in fulldetail_faults.columns if 'spn_fmi' in col] 

    # rolling window function over faults - by default just taking IF a code appears in a 24 hr past window
    faults_rolling = (
        fulldetail_faults
            .groupby('EquipmentID')[faults_cols]
            .rolling(window = time_window, on = "EventTimeStamp")
            .agg(faults_agg)
            .reset_index()
    )
    
    # by default I also decided to apply the same rolling window for the diagnostics part
    # (can be turned off by setting = False, it is quick to execute)
    if windowize_diagnostics:

        # rolling window over diagnostics, by default using mean
        diagnostics_rolling = (
            fulldetail_faults
                .groupby('EquipmentID')[diagnostics_cols]
                .rolling(window = time_window, on = "EventTimeStamp")
                .agg(diagnostics_agg)
                .reset_index()
        )

        # joining back the faults rw to the original dataframe to get the "RecordID" out
        faults_rolling = pd.merge(fulldetail_faults['RecordID'],
                            faults_rolling,
                            left_index= True,
                            right_on = 'level_1').drop(columns='level_1')

        # joining back the diagnostics rw to the original dataframe to get the "RecordID" out
        diagnostics_rolling = pd.merge(fulldetail_faults['RecordID'],
                                diagnostics_rolling,
                                left_index= True,
                                right_on = 'level_1').drop(columns='level_1')
        
        # joining the two rolling windows
        faults_diagnostics_rolling =  pd.merge(
            diagnostics_rolling.drop(columns=['EquipmentID', 'EventTimeStamp']),
            faults_rolling.drop(columns=['EquipmentID', 'EventTimeStamp']),
            on = 'RecordID'
        )

    # this gets used if we only want to take into account the current diagnostics
    # (essentially, NO rolling window for diagnostics)
    else :

        # simply get back 'RecordID' and other diagnostic columns
        faults_diagnostics_rolling = pd.merge(
            fulldetail_faults[['RecordID'] + diagnostics_cols].drop(columns=['EventTimeStamp']),
            faults_rolling.drop(columns=['EquipmentID', 'EventTimeStamp']),
            left_index= True,
            right_on = 'level_1').drop(columns='level_1')
        
    predictor_train = (
        faults_diagnostics_rolling
        .loc[faults_diagnostics_rolling['RecordID']
             .isin(records_train)]
        .sort_values('RecordID')
        .drop(columns='RecordID')
    )
    predictor_test = (
        faults_diagnostics_rolling
        .loc[faults_diagnostics_rolling['RecordID']
             .isin(records_test)]
        .sort_values('RecordID')
        .drop(columns='RecordID')
    )

    return predictor_train, predictor_test

In [66]:
X_train, X_test = windowize_predictors(faults_diagnostics, time_window='1d', faults_agg='max', windowize_diagnostics=False)

In [67]:
gbr = Pipeline(
    steps = [
        ('gb', GradientBoostingClassifier(verbose=True)) #n_estimators = 1000, learning_rate=0.01
    ]
)

In [68]:
gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.0466            6.68m
         2           0.0407            6.72m
         3           0.0397            6.72m
         4           0.0385            6.76m
         5           0.0371            6.66m
         6           0.0363            6.61m
         7           0.0351            6.54m
         8           0.0354            6.45m
         9      281075.3210            6.36m
        10 460581545762231603451447359907642942875092558098495588955744854732510395737689927105143199072518144.0000            6.28m
        20 460581545762231603451447359907642942875092558098495588955744854732510395737689927105143199072518144.0000            5.57m
        30 460581545762231603451447359907642942875092558098495588955744854732510395737689927105143199072518144.0000            4.85m
        40 460581545762231603451447359907642942875092558098495588955744854732510395737689927105143199072518144.0000            4.15m
        50 460581

Pipeline(steps=[('gb', GradientBoostingClassifier(verbose=True))])

In [69]:
print('confusion matrix')
print(confusion_matrix(y_train, gbr.predict(X_train)))
print('\n')
print('classification report')
print(classification_report(y_train, gbr.predict(X_train)))
print('\n')

importances = pd.DataFrame({
    'variable': gbr.feature_names_in_,
    'importance': gbr['gb'].feature_importances_
})

print('Variable Importances:')
display(importances.sort_values('importance', ascending = False).head(20))

print('------ TEST')
print(confusion_matrix(y_test, gbr.predict(X_test)))
print('ROC AUC Score')
print(roc_auc_score(y_true=y_test, y_score=gbr.predict_proba(X_test)[:,1]))

confusion matrix
[[440650    165]
 [  1018    968]]


classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    440815
           1       0.85      0.49      0.62      1986

    accuracy                           1.00    442801
   macro avg       0.93      0.74      0.81    442801
weighted avg       1.00      1.00      1.00    442801



Variable Importances:


,variable,importance
638,spn_fmi_5394_17,0.302671
624,spn_fmi_5246_0,0.106723
2,BarometricPressure,0.090633
4,DistanceLtd,0.089737
467,spn_fmi_4094_18,0.041880
699,spn_fmi_5743_9,0.036837
622,spn_fmi_524287_31,0.031526
16,LampStatus,0.031323
626,spn_fmi_5246_15,0.026365
628,spn_fmi_5246_19,0.024093


------ TEST
[[103372     53]
 [   338    110]]
ROC AUC Score
0.8614902534617908


In [70]:
oversampler = SMOTE(k_neighbors=5, random_state=42)

X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [71]:
# this is going to re-fit from scratch, unless we set warm_start=True
# also, simply add this line to all X_ variables if you want to exclude 5246 influencing the model:
# .drop(columns=[col for col in X_smote.columns if 'spn_fmi_5246' in col])
gbr.fit(X_smote, y_smote)

      Iter       Train Loss   Remaining Time 
         1           1.2506           16.93m
         2           1.1392           16.80m
         3           1.0465           16.55m
         4           0.9684           16.40m
         5           0.9021           16.26m
         6           0.8452           16.11m
         7           0.7963           15.91m
         8           0.7536           15.80m
         9           0.7169           15.68m
        10           0.6828           15.49m
        20           0.4851           13.73m
        30           0.3954           11.56m
        40           0.3517            9.70m
        50           0.3224            7.99m
        60           0.3001            6.34m
        70           0.2804            4.72m
        80           0.2656            3.13m
        90           0.2531            1.56m
       100           0.2434            0.00s


Pipeline(steps=[('gb', GradientBoostingClassifier(verbose=True))])

In [72]:
print('confusion matrix')
print(confusion_matrix(y_train, gbr.predict(X_train)))
print('\n')
print('classification report')
print(classification_report(y_train, gbr.predict(X_train)))
print('\n')

importances = pd.DataFrame({
    'variable': gbr.feature_names_in_,
    'importance': gbr['gb'].feature_importances_
})

print('Variable Importances:')
display(importances.sort_values('importance', ascending = False).head(20))

print('------ TEST')
print('confusion matrix')
print(confusion_matrix(y_test, gbr.predict(X_test)))
print('classification report')
print(classification_report(y_test, gbr.predict(X_test)))
print('ROC AUC Score')
print(roc_auc_score(y_true=y_test, y_score=gbr.predict_proba(X_test)[:,1]))

confusion matrix
[[427160  13655]
 [   226   1760]]


classification report
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    440815
           1       0.11      0.89      0.20      1986

    accuracy                           0.97    442801
   macro avg       0.56      0.93      0.59    442801
weighted avg       1.00      0.97      0.98    442801



Variable Importances:


,variable,importance
163,spn_fmi_1569_31,0.445535
14,FuelTemperature,0.232952
16,LampStatus,0.092798
0,activeTransitionCount,0.038311
300,spn_fmi_3031_9,0.034083
699,spn_fmi_5743_9,0.020157
3,CruiseControlSetSpeed,0.010563
694,spn_fmi_5742_9,0.009019
10,EngineTimeLtd,0.008450
389,spn_fmi_3362_31,0.007591


------ TEST
confusion matrix
[[100420   3005]
 [    82    366]]
classification report
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    103425
           1       0.11      0.82      0.19       448

    accuracy                           0.97    103873
   macro avg       0.55      0.89      0.59    103873
weighted avg       1.00      0.97      0.98    103873

ROC AUC Score
0.9565056955523326


In [73]:
# to load model
# gbr = load('../models/gbr_model_1.joblib') 

# to save model
# dump(gbr, '../models/gbr_model_10.joblib') 

['../models/gbr_model_10.joblib']

Besides saving the models, I will construct a json file that describes how they were obtained.

In [26]:
import json

In [74]:
to_dump = {
    'file_path' : '../models/gbr_model_10.joblib',
    'targets' : 'any row where a derate (5246) happens in the next 24 hours',
    'diagnostics_file' : 'used imputer to average data per truck and then simple mean to average any remaining nulls',
    'train_test_split' : 'using trucks and assuring same ratio of derate and nonderate',
    'windowize_predictors': {'dataframe': 'merged faults and diagnostics',
                             'how far in the past to aggregate' : '1 day (default)',
                             'how to aggregate the one-hot encoded spn_fmi': 'max (default)',
                             'use rolling window on diagnostics?' : 'False ',
                             'how to aggregate diagnostics data' : 'not applicable'},
    'pipeline' : {'step 1': 'GradientBoostingClassifier (default values)'},
    'rebalancing' : {'over or under fitting': 'used SMOTE(k_neighbors=5, random_state=42)',
                     'variables used': 'all (including derate columns)'}

}

tmp_matrix = confusion_matrix(y_train, gbr.predict(X_train))

to_dump['train_confusion_matrix'] = {'TP': int(tmp_matrix[0][0]),
                                     'FP': int(tmp_matrix[0][1]),
                                     'FN': int(tmp_matrix[1][0]),
                                     'TN': int(tmp_matrix[1][1])}

tmp_matrix = confusion_matrix(y_test, gbr.predict(X_test))

to_dump['test_confusion_matrix'] = {'TP': int(tmp_matrix[0][0]),
                                     'FP': int(tmp_matrix[0][1]),
                                     'FN': int(tmp_matrix[1][0]),
                                     'TN': int(tmp_matrix[1][1])}


to_dump['test_rocaouc_score'] = roc_auc_score(y_true=y_test, y_score=gbr.predict_proba(X_test)[:,1])

importances = pd.DataFrame({
    'variable': gbr.feature_names_in_,
    'importance': gbr['gb'].feature_importances_
})

importances = importances.sort_values('importance', ascending = False).head(20)

tmp_dict={}

for index, row in importances.iterrows():
    tmp_dict[row["variable"]] = row['importance']

to_dump['top20_fature_importances'] = tmp_dict


json_object = json.dumps(to_dump, indent=4)

In [75]:
# with open('../models/gbr_model_10.json', 'w') as outfile:
#     outfile.write(json_object)